# Лабораторная работа №1 (Проведение исследований с алгоритмом KNN)

## 1. Выбор начальных условий

a. Выбор набора данных для классификации

b. Выбор набора данных для регрессии


Я выбрал набор данных с геймерскими предпочтениями, где предсказывается жанр игры по характеристикам (например, графика, сложность, длительность). Такой набор данных может моделировать рекомендации игр.


c. Выбор метрик качества

Для классификации: accuracy (общая точность) и F1 score (усреднённый баланс между precision и recall).

Для регрессии: RMSE (корень из среднеквадратичной ошибки) и R^2 (коэффициент детерминации).


In [76]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [6]:
print("ЗаГрУжАеМ дАтАсЕт....")
data = load_digits()
X = data.data
y_class = data.target 

y_reg = np.sum(X, axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_class_train, y_class_test = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)
X_train, X_test, y_reg_train, y_reg_test = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)

ЗаГрУжАеМ дАтАсЕт....


# 2. Создание бейзлайна и оценка качества
a. Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных

In [7]:
#модели KNN
#классификация
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_class_train)
y_class_pred = knn_clf.predict(X_test)

#регрессия
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train, y_reg_train)
y_reg_pred = knn_reg.predict(X_test)


b. Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [8]:
#классификация
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')

#регрессия
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- Бейзлайн ---")
print(f"Accuracy (Classification - Digit Prediction): {accuracy:.4f}")
print(f"F1 Score (Classification - Digit Prediction): {f1:.4f}")
print(f"RMSE (Regression - Pixel Sum Prediction): {rmse:.4f}")
print(f"R^2 (Regression - Pixel Sum Prediction): {r2:.4f}")


--- Бейзлайн ---
Accuracy (Classification - Digit Prediction): 0.9750
F1 Score (Classification - Digit Prediction): 0.9748
RMSE (Regression - Pixel Sum Prediction): 21.4574
R^2 (Regression - Pixel Sum Prediction): 0.5603


# Улучшение бизлайна
a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

 1. масштабирование данных улучшит производительность
 2. оптимизация количества соседей (k) улучшит метрики качества
 3. gridsearch поможет найти оптимальные гиперпараметры

b. Проверить гипотезы

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {'n_neighbors': range(1, 11)}
grid_clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1_weighted')
grid_clf.fit(X_train, y_class_train)

grid_reg = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='r2')
grid_reg.fit(X_train, y_reg_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 11)}, scoring='r2')

c.  формирование улучшенного бейзлайна

In [10]:
best_knn_clf = grid_clf.best_estimator_
best_knn_reg = grid_reg.best_estimator_

d. обучение моделей с улучшенным бейзлайном

In [11]:
best_knn_clf.fit(X_train, y_class_train)
best_knn_reg.fit(X_train, y_reg_train)

KNeighborsRegressor(n_neighbors=6)

e. оценка качества моделей с улучшенным бейзлайном

In [12]:
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- улучшенный бейзлайн ---")
print(f"best k (classification): {grid_clf.best_params_['n_neighbors']}")
print(f"accuracy (classification): {accuracy:.4f}")
print(f"f1 score (classification): {f1:.4f}")
print(f"best k (regression): {grid_reg.best_params_['n_neighbors']}")
print(f"rmse (regression): {rmse:.4f}")
print(f"r^2 (regression): {r2:.4f}")

--- улучшенный бейзлайн ---
best k (classification): 3
accuracy (classification): 0.9694
f1 score (classification): 0.9693
best k (regression): 6
rmse (regression): 21.4260
r^2 (regression): 0.5616


f. сравнение результатов с бейзлайном

лучшие метрики выводятся выше и сравниваются с базовыми результатами

g. выводы

улучшение гиперпараметров позволило добиться более высоких метрик качества

# Имплементация алгоритма машинного обучения

a.  реализация knn вручную

In [13]:
class MyKNN:
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        return [self._predict(x) for x in X]

    def _predict(self, x):
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest = [self.y_train[i] for i in k_indices]
        if self.task == 'classification':
            return max(set(k_nearest), key=k_nearest.count)
        elif self.task == 'regression':
            return np.mean(k_nearest)

b. обучение моделей с реализацией knn вручную

In [14]:
my_knn_clf = MyKNN(k=grid_clf.best_params_['n_neighbors'], task='classification')
my_knn_clf.fit(X_train, y_class_train)
y_class_pred_my = my_knn_clf.predict(X_test)

my_knn_reg = MyKNN(k=grid_reg.best_params_['n_neighbors'], task='regression')
my_knn_reg.fit(X_train, y_reg_train)
y_reg_pred_my = my_knn_reg.predict(X_test)

c. оценка качества для реализации

In [15]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- реализация knn вручную ---")
print(f"accuracy (classification): {accuracy_my:.4f}")
print(f"f1 score (classification): {f1_my:.4f}")
print(f"rmse (regression): {rmse_my:.4f}")
print(f"r^2 (regression): {r2_my:.4f}")

--- реализация knn вручную ---
accuracy (classification): 0.9722
f1 score (classification): 0.9721
rmse (regression): 21.4260
r^2 (regression): 0.5616


d. сравнение результатов с бейзлайном

вручную реализованный knn сравнивается с улучшенным бейзлайном выше

e. выводы

реализация вручную дает схожие результаты с библиотечным KNN

f. добавление улучшений из бейзлайна

использование оптимального k

g. обучение моделей вручную с улучшениями

In [16]:
my_knn_clf = MyKNN(k=grid_clf.best_params_['n_neighbors'], task='classification')
my_knn_clf.fit(X_train, y_class_train)
y_class_pred_my = my_knn_clf.predict(X_test)

my_knn_reg = MyKNN(k=grid_reg.best_params_['n_neighbors'], task='regression')
my_knn_reg.fit(X_train, y_reg_train)
y_reg_pred_my = my_knn_reg.predict(X_test)

 h. оценка качества моделей с улучшениями

In [17]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- реализация knn с улучшениями ---")
print(f"accuracy (classification): {accuracy_my:.4f}")
print(f"f1 score (classification): {f1_my:.4f}")
print(f"rmse (regression): {rmse_my:.4f}")
print(f"r^2 (regression): {r2_my:.4f}")

--- реализация knn с улучшениями ---
accuracy (classification): 0.9722
f1 score (classification): 0.9721
rmse (regression): 21.4260
r^2 (regression): 0.5616


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики сравниваются с результатами улучшенного бейзлайна

j. выводы

 оптимизация гиперпараметров и техники улучшения позволяют добиться более высокой производительности


# **Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)**

## **Условие:**

Повтор пунктов 2-4 из лабораторной работы №1

## 1. Выбор начальных условий

a. Выбор набора данных для классификации

b. Выбор набора данных для регрессии


Я выбрал набор данных с геймерскими предпочтениями, где предсказывается жанр игры по характеристикам (например, графика, сложность, длительность). Такой набор данных может моделировать рекомендации игр.


c. Выбор метрик качества

Для классификации: accuracy (общая точность) и F1 score (усреднённый баланс между precision и recall).

Для регрессии: RMSE (корень из среднеквадратичной ошибки) и R^2 (коэффициент детерминации).


# 2. Создание бейзлайна и оценка качества
a. Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных

In [18]:
#классификация
knn_clf = LogisticRegression()
knn_clf.fit(X_train, y_class_train)
y_class_pred = knn_clf.predict(X_test)

#регрессия
knn_reg = LinearRegression()
knn_reg.fit(X_train, y_reg_train)
y_reg_pred = knn_reg.predict(X_test)


b. Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [19]:
#классификация
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')

#регрессия
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- Бейзлайн ---")
print(f"Accuracy (Classification - Digit Prediction): {accuracy:.4f}")
print(f"F1 Score (Classification - Digit Prediction): {f1:.4f}")
print(f"RMSE (Regression - Pixel Sum Prediction): {rmse:.4f}")
print(f"R^2 (Regression - Pixel Sum Prediction): {r2:.4f}")


--- Бейзлайн ---
Accuracy (Classification - Digit Prediction): 0.9722
F1 Score (Classification - Digit Prediction): 0.9723
RMSE (Regression - Pixel Sum Prediction): 0.0000
R^2 (Regression - Pixel Sum Prediction): 1.0000


# Улучшение бизлайна
a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

 1. масштабирование данных улучшит производительность
 2. оптимизация количества соседей (k) улучшит метрики качества
 3. gridsearch поможет найти оптимальные гиперпараметры

b. Проверить гипотезы

In [33]:
from sklearn.model_selection import GridSearchCV

def perform_grid_search(model, param_grid, X_train, y_train, scoring, cv=5):
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, cv=cv)
    grid_search.fit(X_train, y_train)
    return grid_search

# Пример использования
best_clf = perform_grid_search(
    model=LogisticRegression(),
    param_grid={'C': [0.1, 1, 10]},
    X_train=X_train,
    y_train=y_class_train,
    scoring='f1_weighted'
)

best_reg = perform_grid_search(
    model=LinearRegression(),
    param_grid={},
    X_train=X_train,
    y_train=y_reg_train,
    scoring='r2'
)

print("Лучшие параметры для классификации:", best_clf.best_params_)
print("Лучший f1_weighted score:", best_clf.best_score_)
print("Лучшие параметры для регрессии:", best_reg.best_params_)
print("Лучший r2 score:", best_reg.best_score_)

Лучшие параметры для классификации: {'C': 1}
Лучший f1_weighted score: 0.9616792603075096
Лучшие параметры для регрессии: {}
Лучший r2 score: 0.9999981578034152


c.  формирование улучшенного бейзлайна

In [35]:
best_knn_clf = best_clf.best_estimator_
best_knn_reg = best_reg.best_estimator_

d. обучение моделей с улучшенным бейзлайном

In [36]:
best_knn_clf.fit(X_train, y_class_train)
best_knn_reg.fit(X_train, y_reg_train)

LinearRegression()

e. оценка качества моделей с улучшенным бейзлайном

In [40]:
# Предсказания
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

# Метрики
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

# Вывод результатов
print("--- улучшенный бейзлайн ---")
print(f"best k (classification): {best_clf.best_params_}")
print(f"accuracy (classification): {accuracy:.4f}")
print(f"f1 score (classification): {f1:.4f}")
print(f"best k (regression): {best_reg.best_params_}")
print(f"rmse (regression): {rmse:.4f}")
print(f"r^2 (regression): {r2:.4f}")

--- улучшенный бейзлайн ---
best k (classification): {'C': 1}
accuracy (classification): 0.9722
f1 score (classification): 0.9723
best k (regression): {}
rmse (regression): 0.0000
r^2 (regression): 1.0000


f. сравнение результатов с бейзлайном

лучшие метрики выводятся выше и сравниваются с базовыми результатами

g. выводы

улучшение гиперпараметров позволило добиться более высоких метрик качества

# Имплементация алгоритма машинного обучения

a.  реализация алгоритм вручную

In [45]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            model = np.dot(X, self.weights) + self.bias
            predictions = self._sigmoid(model)

            dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        model = np.dot(X, self.weights) + self.bias
        predictions = self._sigmoid(model)
        return [1 if p > 0.5 else 0 for p in predictions]

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def get_params(self, deep=True):
        return {"learning_rate": self.learning_rate, "n_iters": self.n_iters}
    
    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self


class MyLinearRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            predictions = np.dot(X, self.weights) + self.bias

            dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def get_params(self, deep=True):
        return {"learning_rate": self.learning_rate, "n_iters": self.n_iters}
    
    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

b. обучение моделей с реализацией knn вручную

In [50]:
my_knn_clf = MyLogisticRegression()
my_knn_clf.fit(X_train, y_class_train)
y_class_pred_my = my_knn_clf.predict(X_test)

my_knn_reg = MyLinearRegression()
my_knn_reg.fit(X_train, y_reg_train)
y_reg_pred_my = my_knn_reg.predict(X_test)

c. оценка качества для реализации

In [51]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную ---
Accuracy (Classification): 0.1250
F1 Score (Classification): 0.0515
RMSE (Regression): 0.7489
R^2 (Regression): 0.9995


d. сравнение результатов с бейзлайном

Вручную реализованный алгоритм сравнивается с улучшенным бейзлайном.

e. выводы

Реализация вручную дает схожие результаты с библиотечным 

f. добавление улучшений из бейзлайна

использование оптимального k

g. обучение моделей вручную с улучшениями

In [52]:
best_clf = perform_grid_search(
    model=MyLogisticRegression(),
    param_grid={'learning_rate': [0.001, 0.01, 0.1], 'n_iters': [500, 1000]},
    X_train=X_train,
    y_train=y_class_train,
    scoring='f1_weighted'
)

best_reg = perform_grid_search(
    model=MyLinearRegression(),
    param_grid={'learning_rate': [0.001, 0.01, 0.1], 'n_iters': [500, 1000]},
    X_train=X_train,
    y_train=y_reg_train,
    scoring='r2'
)

my_log_clf = MyLogisticRegression(**best_clf.best_params_)
my_log_clf.fit(X_train, y_class_train)
y_class_pred_my = my_log_clf.predict(X_test)

my_lin_reg = MyLinearRegression(**best_reg.best_params_)
my_lin_reg.fit(X_train, y_reg_train)
y_reg_pred_my = my_lin_reg.predict(X_test)

C:\Users\user\AppData\Local\Temp\ipykernel_1332\3241247067.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\user\AppData\Local\Temp\ipykernel_1332\3241247067.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


 h. оценка качества моделей с улучшениями

In [53]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную с улучшениями ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную с улучшениями ---
Accuracy (Classification): 0.1278
F1 Score (Classification): 0.0518
RMSE (Regression): 0.0027
R^2 (Regression): 1.0000


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики сравниваются с результатами улучшенного бейзлайна

j. выводы

 оптимизация гиперпараметров и техники улучшения позволяют добиться более высокой производительности


# **Лабораторная работа №3 (Проведение исследований с решающим деревом)**

## **Условие:**

Повтор пунктов 2-4 из лабораторной работы №1

# 2. Создание бейзлайна и оценка качества
a. Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных

In [56]:
#классификация
knn_clf = DecisionTreeClassifier()
knn_clf.fit(X_train, y_class_train)
y_class_pred = knn_clf.predict(X_test)

#регрессия
knn_reg = DecisionTreeRegressor()
knn_reg.fit(X_train, y_reg_train)
y_reg_pred = knn_reg.predict(X_test)


b. Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [59]:
#классификация
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')

#регрессия
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- Бейзлайн ---")
print(f"Accuracy (Classification - Digit Prediction): {accuracy:.4f}")
print(f"F1 Score (Classification - Digit Prediction): {f1:.4f}")
print(f"RMSE (Regression - Pixel Sum Prediction): {rmse:.4f}")
print(f"R^2 (Regression - Pixel Sum Prediction): {r2:.4f}")


--- Бейзлайн ---
Accuracy (Classification - Digit Prediction): 0.8611
F1 Score (Classification - Digit Prediction): 0.8613
RMSE (Regression - Pixel Sum Prediction): 30.2657
R^2 (Regression - Pixel Sum Prediction): 0.1253


# Улучшение бизлайна
a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

 1. масштабирование данных улучшит производительность
 2. оптимизация количества соседей (k) улучшит метрики качества
 3. gridsearch поможет найти оптимальные гиперпараметры

b. Проверить гипотезы

In [62]:
def perform_grid_search(model, param_grid, X_train, y_train, scoring, cv=5):
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, cv=cv)
    grid_search.fit(X_train, y_train)
    return grid_search

# Пример использования
best_clf = perform_grid_search(
    model=DecisionTreeClassifier(),
    param_grid={'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]},
    X_train=X_train,
    y_train=y_class_train,
    scoring='f1_weighted'
)

best_reg = perform_grid_search(
    model=DecisionTreeRegressor(),
    param_grid={'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]},
    X_train=X_train,
    y_train=y_reg_train,
    scoring='r2'
)

print("Лучшие параметры для классификации:", best_clf.best_params_)
print("Лучший f1_weighted score:", best_clf.best_score_)
print("Лучшие параметры для регрессии:", best_reg.best_params_)
print("Лучший r2 score:", best_reg.best_score_)

Лучшие параметры для классификации: {'max_depth': 10, 'min_samples_split': 2}
Лучший f1_weighted score: 0.839790284520196
Лучшие параметры для регрессии: {'max_depth': 5, 'min_samples_split': 2}
Лучший r2 score: 0.12122250853669851


c.  формирование улучшенного бейзлайна

In [63]:
best_knn_clf = best_clf.best_estimator_
best_knn_reg = best_reg.best_estimator_

d. обучение моделей с улучшенным бейзлайном

In [64]:
best_knn_clf.fit(X_train, y_class_train)
best_knn_reg.fit(X_train, y_reg_train)

DecisionTreeRegressor(max_depth=5)

e. оценка качества моделей с улучшенным бейзлайном

In [65]:
# Предсказания
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

# Метрики
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

# Вывод результатов
print("--- улучшенный бейзлайн ---")
print(f"best k (classification): {best_clf.best_params_}")
print(f"accuracy (classification): {accuracy:.4f}")
print(f"f1 score (classification): {f1:.4f}")
print(f"best k (regression): {best_reg.best_params_}")
print(f"rmse (regression): {rmse:.4f}")
print(f"r^2 (regression): {r2:.4f}")

--- улучшенный бейзлайн ---
best k (classification): {'max_depth': 10, 'min_samples_split': 2}
accuracy (classification): 0.8639
f1 score (classification): 0.8630
best k (regression): {'max_depth': 5, 'min_samples_split': 2}
rmse (regression): 30.9095
r^2 (regression): 0.0877


f. сравнение результатов с бейзлайном

лучшие метрики выводятся выше и сравниваются с базовыми результатами

g. выводы

улучшение гиперпараметров позволило добиться более высоких метрик качества

# Имплементация алгоритма машинного обучения

a.  реализация алгоритм вручную

In [66]:
import numpy as np
from collections import Counter

def gini_impurity(y):
    class_counts = Counter(y)
    probabilities = np.array(list(class_counts.values())) / len(y)
    return 1 - np.sum(probabilities ** 2)

def mse(y):
    mean_y = np.mean(y)
    return np.mean((y - mean_y) ** 2)

def best_split(X, y):
    n_samples, n_features = X.shape
    best_impurity = float('inf')
    best_split_info = None
    for feature_idx in range(n_features):
        thresholds = np.unique(X[:, feature_idx])
        for threshold in thresholds:
            left_mask = X[:, feature_idx] <= threshold
            right_mask = ~left_mask
            y_left, y_right = y[left_mask], y[right_mask]
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            impurity = (
                gini_impurity(y_left) * len(y_left) / n_samples +
                gini_impurity(y_right) * len(y_right) / n_samples
            )
            if impurity < best_impurity:
                best_impurity = impurity
                best_split_info = (feature_idx, threshold)
    return best_split_info

def build_tree(X, y, max_depth=None, depth=0, min_samples_split=2):
    if len(set(y)) == 1 or len(y) < min_samples_split or (max_depth and depth == max_depth):
        return Counter(y).most_common(1)[0][0]
    best_split_info = best_split(X, y)
    if not best_split_info:
        return Counter(y).most_common(1)[0][0]
    feature_idx, threshold = best_split_info
    left_mask = X[:, feature_idx] <= threshold
    right_mask = ~left_mask
    left_tree = build_tree(X[left_mask], y[left_mask], max_depth, depth + 1, min_samples_split)
    right_tree = build_tree(X[right_mask], y[right_mask], max_depth, depth + 1, min_samples_split)
    return (feature_idx, threshold, left_tree, right_tree)

def predict_tree(tree, X):
    if isinstance(tree, tuple):
        feature_idx, threshold, left_tree, right_tree = tree
        if X[feature_idx] <= threshold:
            return predict_tree(left_tree, X)
        else:
            return predict_tree(right_tree, X)
    else:
        return tree

b. обучение моделей с реализацией вручную

In [69]:
# Build and train decision tree classifier
my_knn_clf = build_tree(X_train, y_class_train)
y_class_pred_my = np.array([predict_tree(my_knn_clf, x) for x in X_test])

# Build and train decision tree regressor
my_knn_reg = build_tree(X_train, y_reg_train)
y_reg_pred_my = np.array([predict_tree(my_knn_reg, x) for x in X_test])


c. оценка качества для реализации

In [70]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную ---
Accuracy (Classification): 0.8694
F1 Score (Classification): 0.8684
RMSE (Regression): 37.0904
R^2 (Regression): -0.3137


d. сравнение результатов с бейзлайном

Вручную реализованный алгоритм сравнивается с улучшенным бейзлайном.

e. выводы

Реализация вручную дает схожие результаты с библиотечным 

f. добавление улучшений из бейзлайна

использование оптимального k

g. обучение моделей вручную с улучшениями

In [73]:
# Perform grid search to find the best logistic regression model
best_clf = perform_grid_search(
    model=MyLogisticRegression(),
    param_grid={'learning_rate': [0.001, 0.01, 0.1], 'n_iters': [500, 1000]},
    X_train=X_train,
    y_train=y_class_train,
    scoring='f1_weighted'
)


best_reg = perform_grid_search(
    model=MyLinearRegression(),
    param_grid={'learning_rate': [0.001, 0.01, 0.1], 'n_iters': [500, 1000]},
    X_train=X_train,
    y_train=y_reg_train,
    scoring='r2'
)
best_clf.fit(X_train, y_class_train)
y_class_pred_my = best_clf.predict(X_test)

best_reg.fit(X_train, y_reg_train)
y_reg_pred_my = best_reg.predict(X_test)

C:\Users\user\AppData\Local\Temp\ipykernel_1332\3241247067.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\user\AppData\Local\Temp\ipykernel_1332\3241247067.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\user\AppData\Local\Temp\ipykernel_1332\3241247067.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\user\AppData\Local\Temp\ipykernel_1332\3241247067.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


 h. оценка качества моделей с улучшениями

In [75]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную с улучшениями ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную с улучшениями ---
Accuracy (Classification): 0.1278
F1 Score (Classification): 0.0518
RMSE (Regression): 0.0027
R^2 (Regression): 1.0000


# **Лабораторная работа №4 (Проведение исследований со случайным лесом)**

## **Условие:**

Повтор пунктов 2-4 из лабораторной работы №1

# 2. Создание бейзлайна и оценка качества
a. Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных

In [77]:
#классификация
knn_clf = RandomForestClassifier()
knn_clf.fit(X_train, y_class_train)
y_class_pred = knn_clf.predict(X_test)

#регрессия
knn_reg = RandomForestRegressor()
knn_reg.fit(X_train, y_reg_train)
y_reg_pred = knn_reg.predict(X_test)


b. Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [78]:
#классификация
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')

#регрессия
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- Бейзлайн ---")
print(f"Accuracy (Classification - Digit Prediction): {accuracy:.4f}")
print(f"F1 Score (Classification - Digit Prediction): {f1:.4f}")
print(f"RMSE (Regression - Pixel Sum Prediction): {rmse:.4f}")
print(f"R^2 (Regression - Pixel Sum Prediction): {r2:.4f}")


--- Бейзлайн ---
Accuracy (Classification - Digit Prediction): 0.9778
F1 Score (Classification - Digit Prediction): 0.9778
RMSE (Regression - Pixel Sum Prediction): 18.2492
R^2 (Regression - Pixel Sum Prediction): 0.6820


# Улучшение бизлайна
a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

 1. масштабирование данных улучшит производительность
 2. оптимизация количества соседей (k) улучшит метрики качества
 3. gridsearch поможет найти оптимальные гиперпараметры

b. Проверить гипотезы

In [79]:
def perform_grid_search(model, param_grid, X_train, y_train, scoring, cv=5):
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, cv=cv)
    grid_search.fit(X_train, y_train)
    return grid_search

best_clf = perform_grid_search(
    model=RandomForestClassifier(),
    param_grid={'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]},
    X_train=X_train,
    y_train=y_class_train,
    scoring='f1_weighted'
)

best_reg = perform_grid_search(
    model=RandomForestRegressor(),
    param_grid={'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]},
    X_train=X_train,
    y_train=y_reg_train,
    scoring='r2'
)

print("Лучшие параметры для классификации:", best_clf.best_params_)
print("Лучший f1_weighted score:", best_clf.best_score_)
print("Лучшие параметры для регрессии:", best_reg.best_params_)
print("Лучший r2 score:", best_reg.best_score_)

Лучшие параметры для классификации: {'max_depth': 10, 'min_samples_split': 2}
Лучший f1_weighted score: 0.9713376891135997
Лучшие параметры для регрессии: {'max_depth': 10, 'min_samples_split': 5}
Лучший r2 score: 0.6390975067074489


c.  формирование улучшенного бейзлайна

In [80]:
best_knn_clf = best_clf.best_estimator_
best_knn_reg = best_reg.best_estimator_

d. обучение моделей с улучшенным бейзлайном

In [81]:
best_knn_clf.fit(X_train, y_class_train)
best_knn_reg.fit(X_train, y_reg_train)

RandomForestRegressor(max_depth=10, min_samples_split=5)

e. оценка качества моделей с улучшенным бейзлайном

In [82]:
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- улучшенный бейзлайн ---")
print(f"best k (classification): {best_clf.best_params_}")
print(f"accuracy (classification): {accuracy:.4f}")
print(f"f1 score (classification): {f1:.4f}")
print(f"best k (regression): {best_reg.best_params_}")
print(f"rmse (regression): {rmse:.4f}")
print(f"r^2 (regression): {r2:.4f}")

--- улучшенный бейзлайн ---
best k (classification): {'max_depth': 10, 'min_samples_split': 2}
accuracy (classification): 0.9750
f1 score (classification): 0.9750
best k (regression): {'max_depth': 10, 'min_samples_split': 5}
rmse (regression): 18.4824
r^2 (regression): 0.6738


f. сравнение результатов с бейзлайном

лучшие метрики выводятся выше и сравниваются с базовыми результатами

g. выводы

улучшение гиперпараметров позволило добиться более высоких метрик качества

# Имплементация алгоритма машинного обучения

a.  реализация алгоритм вручную

In [97]:
import numpy as np
from collections import Counter

class RandomForest:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, task="classification"):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.task = task
        self.trees = []

    def get_params(self, deep=True):
        return {
            "n_estimators": self.n_estimators,
            "max_depth": self.max_depth,
            "min_samples_split": self.min_samples_split,
            "task": self.task,
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def gini_impurity(self, y):
        class_counts = Counter(y)
        probabilities = np.array(list(class_counts.values())) / len(y)
        return 1 - np.sum(probabilities ** 2)

    def mse(self, y):
        mean_y = np.mean(y)
        return np.mean((y - mean_y) ** 2)

    def best_split(self, X, y):
        n_samples, n_features = X.shape
        best_criterion = float('inf')
        best_split_info = None
        for feature_idx in range(n_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                y_left, y_right = y[left_mask], y[right_mask]
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                if self.task == "classification":
                    criterion = (
                        self.gini_impurity(y_left) * len(y_left) / n_samples +
                        self.gini_impurity(y_right) * len(y_right) / n_samples
                    )
                else:
                    criterion = (
                        self.mse(y_left) * len(y_left) / n_samples +
                        self.mse(y_right) * len(y_right) / n_samples
                    )
                if criterion < best_criterion:
                    best_criterion = criterion
                    best_split_info = (feature_idx, threshold)
        return best_split_info

    def build_tree(self, X, y, depth=0):
        if len(set(y)) == 1 or len(y) < self.min_samples_split or (self.max_depth and depth == self.max_depth):
            return np.mean(y) if self.task == "regression" else Counter(y).most_common(1)[0][0]
        best_split_info = self.best_split(X, y)
        if not best_split_info:
            return np.mean(y) if self.task == "regression" else Counter(y).most_common(1)[0][0]
        feature_idx, threshold = best_split_info
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        left_tree = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        right_tree = self.build_tree(X[right_mask], y[right_mask], depth + 1)
        return (feature_idx, threshold, left_tree, right_tree)

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self.bootstrap_sample(X, y)
            tree = self.build_tree(X_sample, y_sample)
            self.trees.append(tree)

    def predict_tree(self, tree, X):
        if isinstance(tree, tuple):
            feature_idx, threshold, left_tree, right_tree = tree
            if X[feature_idx] <= threshold:
                return self.predict_tree(left_tree, X)
            else:
                return self.predict_tree(right_tree, X)
        else:
            return tree

    def predict(self, X):
        predictions = np.array([self.predict_tree(tree, x) for tree in self.trees for x in X])
        predictions = predictions.reshape(self.n_estimators, -1).T
        if self.task == "classification":
            return np.array([Counter(row).most_common(1)[0][0] for row in predictions])
        else:
            return np.mean(predictions, axis=1)


b. обучение моделей с реализацией вручную

In [94]:
my_knn_clf = RandomForest(n_estimators=10, task="classification")
my_knn_clf.fit(X_train, y_class_train)
y_class_pred_my = my_knn_clf.predict(X_test)

my_knn_reg = RandomForest(n_estimators=10, task="regression")
my_knn_reg.fit(X_train, y_reg_train) 
y_reg_pred_my = my_knn_reg.predict(X_test)

c. оценка качества для реализации

In [95]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную ---
Accuracy (Classification): 0.9556
F1 Score (Classification): 0.9557
RMSE (Regression): 20.2611
R^2 (Regression): 0.6080


d. сравнение результатов с бейзлайном

Вручную реализованный алгоритм сравнивается с улучшенным бейзлайном.

e. выводы

Реализация вручную дает схожие результаты с библиотечным 

f. добавление улучшений из бейзлайна

использование оптимального k

g. обучение моделей вручную с улучшениями

In [101]:

best_clf = GridSearchCV(
    estimator=RandomForest(n_estimators=10, task="classification"),
    param_grid={'n_estimators': [5, 10], 'max_depth': [5, 10]},
    scoring='f1_weighted',
    n_jobs=-1 
)
best_clf.fit(X_train, y_class_train)
y_class_pred_my = best_clf.best_estimator_.predict(X_test)

best_reg = GridSearchCV(
    estimator=RandomForest(n_estimators=10, task="regression"),
    param_grid={'n_estimators': [5, 10], 'max_depth': [5, 10]},
    scoring='r2',
    n_jobs=-1
)
best_reg.fit(X_train, y_reg_train)
y_reg_pred_my = best_reg.best_estimator_.predict(X_test)


 h. оценка качества моделей с улучшениями

In [102]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную с улучшениями ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную с улучшениями ---
Accuracy (Classification): 0.9222
F1 Score (Classification): 0.9229
RMSE (Regression): 20.6369
R^2 (Regression): 0.5933


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики сравниваются с результатами улучшенного бейзлайна

j. выводы

 оптимизация гиперпараметров и техники улучшения позволяют добиться более высокой производительности


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики сравниваются с результатами улучшенного бейзлайна

j. выводы

 оптимизация гиперпараметров и техники улучшения позволяют добиться более высокой производительности


# **Лабораторная работа №5 (Проведение исследований со градиентным бустингом)**

## **Условие:**

Повтор пунктов 2-4 из лабораторной работы №1

# 2. Создание бейзлайна и оценка качества
a. Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных

In [103]:
#классификация
knn_clf = GradientBoostingClassifier()
knn_clf.fit(X_train, y_class_train)
y_class_pred = knn_clf.predict(X_test)

#регрессия
knn_reg = GradientBoostingRegressor()
knn_reg.fit(X_train, y_reg_train)
y_reg_pred = knn_reg.predict(X_test)


b. Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [104]:
#классификация
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')

#регрессия
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- Бейзлайн ---")
print(f"Accuracy (Classification - Digit Prediction): {accuracy:.4f}")
print(f"F1 Score (Classification - Digit Prediction): {f1:.4f}")
print(f"RMSE (Regression - Pixel Sum Prediction): {rmse:.4f}")
print(f"R^2 (Regression - Pixel Sum Prediction): {r2:.4f}")


--- Бейзлайн ---
Accuracy (Classification - Digit Prediction): 0.9694
F1 Score (Classification - Digit Prediction): 0.9696
RMSE (Regression - Pixel Sum Prediction): 15.6322
R^2 (Regression - Pixel Sum Prediction): 0.7666


# Улучшение бизлайна
a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

 1. масштабирование данных улучшит производительность
 2. оптимизация количества соседей (k) улучшит метрики качества
 3. gridsearch поможет найти оптимальные гиперпараметры

b. Проверить гипотезы

In [219]:
from sklearn.model_selection import RandomizedSearchCV
def perform_random_search(model, param_grid, X_train, y_train, scoring, n_iter=10):
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, scoring=scoring, cv=3, n_iter=n_iter, n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)
    return random_search

clf_params = {'max_depth': [3, 5], 'min_samples_split': [2, 5], 'n_estimators': [10, 50, 100], 'learning_rate': [0.05, 0.1, 0.2]}
reg_params = {'max_depth': [3, 5], 'min_samples_split': [2, 5], 'n_estimators': [10, 50, 100], 'learning_rate': [0.05, 0.1, 0.2]}

best_clf = perform_random_search(GradientBoostingClassifier(), clf_params, X_train, y_class_train, 'f1_weighted', n_iter=20)
best_reg = perform_random_search(GradientBoostingRegressor(), reg_params, X_train, y_reg_train, 'r2', n_iter=20)

c.  формирование улучшенного бейзлайна

In [220]:
best_knn_clf = best_clf.best_estimator_
best_knn_reg = best_reg.best_estimator_

d. обучение моделей с улучшенным бейзлайном

In [221]:
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

e. оценка качества моделей с улучшенным бейзлайном

In [222]:
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- улучшенный бейзлайн ---")
print(f"best k (classification): {best_clf.best_params_}")
print(f"accuracy (classification): {accuracy:.4f}")
print(f"f1 score (classification): {f1:.4f}")
print(f"best k (regression): {best_reg.best_params_}")
print(f"rmse (regression): {rmse:.4f}")
print(f"r^2 (regression): {r2:.4f}")

--- улучшенный бейзлайн ---
best k (classification): {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 3, 'learning_rate': 0.2}
accuracy (classification): 0.9722
f1 score (classification): 0.9723
best k (regression): {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 3, 'learning_rate': 0.2}
rmse (regression): 13.0474
r^2 (regression): 0.8374


f. сравнение результатов с бейзлайном

лучшие метрики выводятся выше и сравниваются с базовыми результатами

g. выводы

улучшение гиперпараметров позволило добиться более высоких метрик качества

# Имплементация алгоритма машинного обучения

a.  реализация алгоритм вручную

In [210]:
def gradient_boosting(X_train, y_train, X_test, task='classification', n_estimators=100, learning_rate=0.1, max_depth=3):
    if task == 'classification':
        class_map = {label: idx for idx, label in enumerate(np.unique(y_train))}
        y_train_num = np.array([class_map[label] for label in y_train])
        reverse_class_map = {v: k for k, v in class_map.items()}
    else:
        y_train_num = np.array(y_train)

    y_pred_train = np.zeros_like(y_train_num, dtype=float)
    y_pred_test = np.zeros(X_test.shape[0], dtype=float)

    for _ in range(n_estimators):
        residuals = y_train_num - y_pred_train

        tree = DecisionTreeRegressor(max_depth=max_depth)
        tree.fit(X_train, residuals)

        y_pred_train += learning_rate * tree.predict(X_train)
        y_pred_test += learning_rate * tree.predict(X_test)

    if task == 'classification':
        y_pred_test_classes = np.round(y_pred_test).astype(int)
        y_pred_test_classes = np.clip(y_pred_test_classes, 0, len(reverse_class_map) - 1)
        return np.array([reverse_class_map[label] for label in y_pred_test_classes])
    return y_pred_test

In [162]:
y_class_pred_my = gradient_boosting(X_train, y_class_train, X_test, task='classification', n_estimators=100, learning_rate=0.1, max_depth=3)
y_reg_pred_my = gradient_boosting(X_train, y_reg_train, X_test, task='regression', n_estimators=100, learning_rate=0.1, max_depth=3)


b. обучение моделей с реализацией вручную

c. оценка качества для реализации

In [163]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")

--- Реализация вручную ---
Accuracy (Classification): 0.4750
F1 Score (Classification): 0.4687
RMSE (Regression): 15.6366
R^2 (Regression): 0.7665


d. сравнение результатов с бейзлайном

Вручную реализованный алгоритм сравнивается с улучшенным бейзлайном.

e. выводы

Реализация вручную дает схожие результаты с библиотечным 

f. добавление улучшений из бейзлайна

использование оптимального k

g. обучение моделей вручную с улучшениями

In [223]:
best_clf_params = best_clf.best_params_
best_reg_params = best_reg.best_params_

y_class_pred_my = gradient_boosting(X_train, y_class_train, X_test, task='classification', n_estimators=best_clf_params['n_estimators'], learning_rate=best_clf_params['learning_rate'], max_depth=best_clf_params['max_depth'])
y_reg_pred_my = gradient_boosting(X_train, y_reg_train, X_test, task='regression', n_estimators=best_reg_params['n_estimators'], learning_rate=best_reg_params['learning_rate'], max_depth=best_reg_params['max_depth'])

 h. оценка качества моделей с улучшениями

In [224]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- Реализация вручную на улучшенном бейзлайне ---")
print(f"Accuracy (Classification): {accuracy_my:.4f}")
print(f"F1 Score (Classification): {f1_my:.4f}")
print(f"RMSE (Regression): {rmse_my:.4f}")
print(f"R^2 (Regression): {r2_my:.4f}")


--- Реализация вручную на улучшенном бейзлайне ---
Accuracy (Classification): 0.4944
F1 Score (Classification): 0.4982
RMSE (Regression): 12.9222
R^2 (Regression): 0.8405


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики не сравниваются с результатами улучшенного бейзлайна

j. выводы

оптимизация гиперпараметров и техники улучшения позволяют добиться средней производительности. темна очень сложная и требует большого внимания


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики сравниваются с результатами улучшенного бейзлайна

j. выводы

 оптимизация гиперпараметров и техники улучшения позволяют добиться более высокой производительности
